In [1]:
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import ArxivAPIWrapper, WikipediaAPIWrapper

In [2]:
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

'wikipedia'

In [3]:
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=250)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
print(arxiv.name)

arxiv


In [4]:
tools = [wiki, arxiv]

In [5]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
# from langchain_community.embeddings import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
embeddings = OllamaEmbeddings(model="llama3.2:latest")

/var/folders/qk/8pjm3ryn39g988_lw7jsnx9m0000gn/T/ipykernel_35527/3309661420.py:1: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="llama3.2:latest")


In [7]:
loader = WebBaseLoader("https://docs.smith.langchain.com")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200).split_documents(docs)
vectordb = FAISS.from_documents(documents, embeddings)
retriever = vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x10af5f280>, search_kwargs={})

In [8]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever, "langsmith-search", "Search any information about Langsmith")
retriever_tool.name

'langsmith-search'

In [9]:
tools = [arxiv, wiki, retriever_tool]

In [10]:
tools

[ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/Users/adityameshram/cpp/SearchEngine/.conda/lib/python3.10/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith-search', description='Search any information about Langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x10ab5d240>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x10af5f280>, 

In [11]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()
import os

groq_api_key = os.getenv("GROQ_API_KEY")
llm = ChatGroq(groq_api_key=groq_api_key, model_name = "Llama3-8b-8192")

In [12]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage

prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are a helpful assistant."),
    HumanMessagePromptTemplate.from_template("{input}")
])

In [13]:
# from langchain.agents import create_openapi_tools_agent
# agent = create_openapi_tools_agent(llm, tools, prompt)
# agent

In [14]:
# from langchain.agents import AgentExecutor
# agent_executer = AgentExecutor(agent=agent, tools = tools, verbose=True)
# agent_executer

In [15]:
# agent_executer.invoke({"input" : "Tell me about Langsmith"})

In [17]:
from langchain.agents import initialize_agent
from langchain.agents.agent_types import AgentType

agent_executor = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,  # works with Groq
    verbose=True
)

response = agent_executor.invoke({
    "input": "Tell me about Langsmith",
    "chat_history": []  # Empty history
})
print(response)



> Entering new AgentExecutor chain...
```json
{
    "action": "langsmith-search",
    "action_input": "Langsmith"
}
```
Observation: Get started with LangSmith | ü¶úÔ∏èüõ†Ô∏è LangSmith

For more see the how-to guide for setting up LangSmith with LangChain or setting up LangSmith with LangGraph.
Observability‚Äã
Observability is important for any software application, but especially so for LLM applications. LLMs are non-deterministic by nature, meaning they can produce unexpected results. This makes them trickier than normal to debug.
This is where LangSmith can help! LangSmith has LLM-native observability, allowing you to get meaningful insights from your application. LangSmith‚Äôs observability features have you covered throughout all stages of application development - from prototyping, to beta testing, to production.

Get started by adding tracing to your application.
Create dashboards to view key metrics like RPS, error rates and costs.

Evals‚Äã
The quality and development spe

In [18]:
response = agent_executor.invoke({
    "input": "What is ML",
    "chat_history": []  # Empty history
})
print(response)



> Entering new AgentExecutor chain...
```json
{
    "action": "wikipedia",
    "action_input": "Machine learning"
}
```
Observation: Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks wit
Thought:```json
{
    "action": "Final Answer",
    "action_input": "Machine learning is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks without being explicitly programmed."
}

> Finished chain.
{'input': 'What is ML', 'chat_history': [], 'output': 'Machine learning is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks without bei

In [19]:
response = agent_executor.invoke({
    "input": "arXiv:1706.03762",
    "chat_history": []  # Empty history
})
print(response)	 



> Entering new AgentExecutor chain...
```json
{
    "action": "arxiv",
    "action_input": "1706.03762"
}
```
Observation: Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on c
Thought:```json
{
    "action": "Final Answer",
    "action_input": "The response is about the paper titled 'Attention Is All You Need' published on 2023-08-02, which is a research paper by Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, and Illia Polosukhin. The paper discusses sequence transduction models."
}

> Finished chain.
{'input': 'arXiv:1706.03762', 'chat_history': [], 'output': "The response is about the paper titled 'Attention Is All You Need' published on 2023-08-02, which is a research paper by Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszko